In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# 3.2.5 Random Forest Guided Example

We've talked about Random Forests. Now it's time to build one.

Here we'll use data from Lending Club to predict the state of a loan given some information about it. You can find the dataset [here](https://www.lendingclub.com/info/download-data.action). We'll use 2015 data. ([Thinkful mirror](https://www.dropbox.com/s/m7z42lubaiory33/LoanStats3d.csv?dl=0))

In [2]:
# Replace the path with the correct path for your data.
y2015 = pd.read_csv(
    'LoanStats3d.csv',
    skipinitialspace=True,
    header=1, nrows=50000
)

# Convert ID and Interest Rate to numeric.
y2015['id'] = pd.to_numeric(y2015['id'], errors='coerce')
y2015['int_rate'] = pd.to_numeric(y2015['int_rate'].str.strip('%'), errors='coerce')

# Drop other columns with many unique variables
y2015.drop(['url', 'emp_title', 'zip_code', 'earliest_cr_line', 'revol_util',
            'sub_grade', 'addr_state', 'desc'], 1, inplace=True)

c:\users\mtracy\appdata\local\programs\python\python36-32\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
y2015.describe()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
count,5.000000e+04,5.000000e+04,50000.000000,50000.000000,50000.00000,50000.000000,50000.000000,5.000000e+04,50000.000000,50000.000000,...,50000.000000,50000.000000,50000.000000,49504.000000,50000.000000,50000.000000,5.000000e+04,50000.000000,50000.000000,50000.000000
mean,6.670962e+07,7.144267e+07,15019.333500,15019.333500,15011.79400,12.236739,434.134945,7.919482e+04,19.540321,0.345620,...,0.093180,2.245820,93.814492,45.537807,0.139320,0.068040,1.773950e+05,53376.550720,22657.988700,45269.696840
std,1.942347e+06,2.243390e+06,8676.180969,8676.180969,8671.20408,4.190234,247.503896,1.013218e+05,45.737896,0.910441,...,0.507644,1.906606,9.038545,36.311435,0.393844,0.455384,1.772051e+05,48872.446963,22498.091256,44708.412285
min,6.702500e+04,2.484710e+05,1000.000000,1000.000000,950.00000,5.320000,30.540000,0.000000e+00,0.000000,0.000000,...,0.000000,0.000000,12.500000,0.000000,0.000000,0.000000,2.500000e+03,0.000000,0.000000,0.000000
25%,6.594684e+07,7.064157e+07,8000.000000,8000.000000,8000.00000,9.170000,255.040000,4.800000e+04,12.680000,0.000000,...,0.000000,1.000000,90.900000,11.100000,0.000000,0.000000,5.270000e+04,23197.000000,8000.000000,16663.750000
50%,6.654286e+07,7.126861e+07,14000.000000,14000.000000,13900.00000,11.990000,378.150000,6.600000e+04,18.820000,0.000000,...,0.000000,2.000000,97.300000,42.900000,0.000000,0.000000,1.152950e+05,40633.500000,15600.000000,34672.000000
75%,6.744762e+07,7.224896e+07,20000.000000,20000.000000,20000.00000,14.480000,573.810000,9.500000e+04,25.620000,0.000000,...,0.000000,3.000000,100.000000,75.000000,0.000000,0.000000,2.554300e+05,67700.000000,29500.000000,60522.000000
max,6.861706e+07,7.354484e+07,35000.000000,35000.000000,35000.00000,28.990000,1354.660000,9.000000e+06,9999.000000,15.000000,...,14.000000,30.000000,100.000000,100.000000,8.000000,25.000000,4.037303e+06,882172.000000,307500.000000,975560.000000


In [4]:
from sklearn import ensemble
from sklearn.model_selection import cross_val_score

rfc = ensemble.RandomForestClassifier()
X = y2015.drop('loan_status', 1)
Y = y2015['loan_status']
X = pd.get_dummies(X)
X = X.dropna(axis=1)

cross_val_score(rfc, X, Y, cv=5)

array([0.98220356, 0.98120188, 0.98110189, 0.9808    , 0.95828331])

### DRILL: So here's your task. Get rid of as much data as possible without dropping below an average of 90% accuracy in a 10-fold cross validation.

You'll want to do a few things in this process. First, dive into the data that we have and see which features are most important. This can be the raw features or the generated dummies. You may want to use PCA or correlation matrices.

Can you do it without using anything related to payment amount or outstanding principal? How do you know?

### PCA 

In [16]:
# Import and run PCA model 
from sklearn.decomposition import PCA
pca = PCA(n_components=10)
pca.fit(X)
X_pca = pca.transform(X)

In [17]:
# Test PCA on forest
rfc = ensemble.RandomForestClassifier()
cross_val_score(rfc, X_pca, Y, cv=10)

array([0.93082767, 0.93722511, 0.93742503, 0.93841232, 0.9440112 ,
       0.9386    , 0.93858772, 0.939988  , 0.93278656, 0.93973974])

Using PCA dimensionality reduction, 90%+ accuracy can be maintained by reducing the data to 10 principle components.  Less than 10 results in <90% accuracy.  

### Can you do it without any information related to payment amount or outstanding principal?

In [27]:
# Examine columns, look for ones related to payment amount or principal
X.columns[:40]

Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'int_rate', 'installment', 'annual_inc', 'dti', 'delinq_2yrs',
       'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal', 'total_acc',
       'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv',
       'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries',
       'collection_recovery_fee', 'last_pymnt_amnt',
       'collections_12_mths_ex_med', 'policy_code', 'acc_now_delinq',
       'tot_coll_amt', 'tot_cur_bal', 'total_rev_hi_lim',
       'acc_open_past_24mths', 'avg_cur_bal', 'chargeoff_within_12_mths',
       'delinq_amnt', 'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op',
       'mo_sin_rcnt_tl', 'mort_acc', 'num_accts_ever_120_pd'],
      dtype='object')

In [26]:
X.columns[41:80]

Index(['num_actv_rev_tl', 'num_bc_sats', 'num_bc_tl', 'num_il_tl',
       'num_op_rev_tl', 'num_rev_accts', 'num_rev_tl_bal_gt_0', 'num_sats',
       'num_tl_30dpd', 'num_tl_90g_dpd_24m', 'num_tl_op_past_12m',
       'pct_tl_nvr_dlq', 'pub_rec_bankruptcies', 'tax_liens',
       'tot_hi_cred_lim', 'total_bal_ex_mort', 'total_bc_limit',
       'total_il_high_credit_limit', 'term_36 months', 'term_60 months',
       'grade_A', 'grade_B', 'grade_C', 'grade_D', 'grade_E', 'grade_F',
       'grade_G', 'emp_length_1 year', 'emp_length_10+ years',
       'emp_length_2 years', 'emp_length_3 years', 'emp_length_4 years',
       'emp_length_5 years', 'emp_length_6 years', 'emp_length_7 years',
       'emp_length_8 years', 'emp_length_9 years', 'emp_length_< 1 year',
       'home_ownership_ANY'],
      dtype='object')

In [28]:
X.columns[81:120]

Index(['home_ownership_OWN', 'home_ownership_RENT',
       'verification_status_Not Verified',
       'verification_status_Source Verified', 'verification_status_Verified',
       'issue_d_Dec-15', 'issue_d_Nov-15', 'pymnt_plan_n', 'purpose_car',
       'purpose_credit_card', 'purpose_debt_consolidation',
       'purpose_home_improvement', 'purpose_house', 'purpose_major_purchase',
       'purpose_medical', 'purpose_moving', 'purpose_other',
       'purpose_renewable_energy', 'purpose_small_business',
       'purpose_vacation', 'title_Business', 'title_Car financing',
       'title_Credit card refinancing', 'title_Debt consolidation',
       'title_Green loan', 'title_Home buying', 'title_Home improvement',
       'title_Major purchase', 'title_Medical expenses',
       'title_Moving and relocation', 'title_Other', 'title_Vacation',
       'initial_list_status_f', 'initial_list_status_w', 'last_pymnt_d_Apr-16',
       'last_pymnt_d_Aug-16', 'last_pymnt_d_Dec-15', 'last_pymnt_d_Dec-16',

In [29]:
X.columns[121:]

Index(['last_pymnt_d_Jan-17', 'last_pymnt_d_Jul-16', 'last_pymnt_d_Jun-16',
       'last_pymnt_d_Mar-16', 'last_pymnt_d_May-16', 'last_pymnt_d_Nov-16',
       'last_pymnt_d_Oct-16', 'last_pymnt_d_Sep-16', 'next_pymnt_d_Feb-17',
       'next_pymnt_d_Jan-17', 'next_pymnt_d_Mar-17',
       'last_credit_pull_d_Apr-16', 'last_credit_pull_d_Aug-16',
       'last_credit_pull_d_Dec-15', 'last_credit_pull_d_Dec-16',
       'last_credit_pull_d_Feb-16', 'last_credit_pull_d_Jan-16',
       'last_credit_pull_d_Jan-17', 'last_credit_pull_d_Jul-16',
       'last_credit_pull_d_Jun-16', 'last_credit_pull_d_Mar-16',
       'last_credit_pull_d_May-16', 'last_credit_pull_d_Nov-15',
       'last_credit_pull_d_Nov-16', 'last_credit_pull_d_Oct-16',
       'last_credit_pull_d_Sep-16', 'application_type_INDIVIDUAL',
       'application_type_JOINT', 'verification_status_joint_Not Verified',
       'verification_status_joint_Source Verified',
       'verification_status_joint_Verified'],
      dtype='object')

In [33]:
# Remove all columns related to payment amount and outstanding principal 
columns = ['out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv',
       'total_rec_prncp', 'total_rec_int','last_pymnt_amnt']

X_trim = X.drop(columns=columns)

# Rerun model with reduced data set
rfc = ensemble.RandomForestClassifier()
cross_val_score(rfc, X_trim, Y, cv=10)

array([0.94822071, 0.95341863, 0.95461815, 0.95480904, 0.95540892,
       0.9568    , 0.95519104, 0.95939188, 0.95639128, 0.95275275])

Yes, accuracy >90% can be achieved if payment amount and outstanding principal information is excluded from the dataset.